In [ ]:
# HW2P2: Image Recognition and Verification

This is the second homework  in 11785: Introduction to Deep Learning. We are trying to tackle the problem of Image Verification. For this, we will need to first train our own CNN model to tackle the problem of classification, consisting of 8631 identities. Using this, we get the face embeddings for different pairs of images and try to identify if the pair of face matches or not.

### For this Assignment of Image Recognition and Verification, I tested different data transformation technique, model Architecture including ResNet, AlexNet, MobileFace, and Residual Squeeze Excite, and different optimizer incFor this assignment on image recognition and verification, I experimented with various data transformation techniques and model architectures, including ResNet, AlexNet, MobileFace, and Residual Squeeze-Excite networks. I also tested different optimizers such as Adam, AdamW, and SGD. Through numerous training runs tracked via WandB, my final experiment, which was 70 epochs, which returned better smaller Equal Error Rate (EER).luding Adam, AdamW,and SGD, I did many runs testing as shown by wandb and my last run was 70 epoch which gave me the good Equal Error Rate (EER) 

# Libraries

In [ ]:
!nvidia-smi # Run this to see what GPU you have

In [ ]:
!pip install wandb --quiet # Install WandB
!pip install pytorch_metric_learning --quiet #Install the Pytorch Metric Library

In [ ]:
!pip install torchsummaryX==1.1.0

In [ ]:
import torch
from torchsummaryX import summary
import torchvision
from torchvision import transforms
import torch.nn.functional as F
import os
import gc
from tqdm import tqdm
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn import metrics as mt
from scipy.optimize import brentq
from scipy.interpolate import interp1d
import glob
import wandb
import matplotlib.pyplot as plt
from pytorch_metric_learning import samplers
import csv

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", DEVICE)


In [ ]:
# from google.colab import drive # Link to your drive if you are not using Colab with GCP
# drive.mount('/content/drive') # Models in this HW take a long time to get trained and make sure to save it here

# Kaggle

### Setting up my kaggle account with my Api key

In [1]:
# TODO: Use the same Kaggle code from HW1P2
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"angeizabayo","key":"6727c97821f5c0aae21046c561c0545d"}')
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73249 sha256=abcd9a4b94b232ffe7aa39606a816d334da4f91a093ee99a478b4ebf2cd5df32
  Stored in directory: /root/.cache/pip/wheels/0b/76/ca/e58f8afa83166a0e68f0d5cd2e7f99d260bdc40e35da080eee
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.6.17
    Uninstalling kaggle-1.6.17:
      Successfully uninstalled kaggle-1.6.17


In [ ]:
# # Reminder: Make sure you have connected your kaggle API before running this block
!mkdir '/kaggle/working'

!kaggle competitions download -c 11785-hw-2-p-2-face-verification-fall-2024
!unzip -qo '11785-hw-2-p-2-face-verification-fall-2024.zip' -d '/kaggle/working'

In [ ]:
!pwd

In [ ]:
!ls /kaggle/working

# Config

In [ ]:
config = {
    'batch_size': 64, # Increase this if your GPU can handle it
    'lr': 0.001,
    'epochs': 70, # 20 epochs is recommended ONLY for the early submission - you will have to train for much longer typically.
    'data_dir': "/kaggle/working/11-785-f24-hw2p2-verification/cls_data", #TODO
    'data_ver_dir': "/kaggle/working/11-785-f24-hw2p2-verification/ver_data", #TODO
    'pair_dir': "/kaggle/working/11-785-f24-hw2p2-verification/",
    'checkpoint_dir': "/kaggle/working/kaggle/working/checkpoint" #TODO
    # Include other parameters as needed.
}


# Dataset

## Dataset Class for doing Image Verification

In [ ]:
class ImagePairDataset(torch.utils.data.Dataset):

    def __init__(self, data_dir, csv_file, transform):
        self.data_dir = data_dir
        self.transform = transform
        self.pairs = []
        if csv_file.endswith('.csv'):
            with open(csv_file, 'r') as f:
                reader = csv.reader(f)
                for i, row in enumerate(reader):
                    if i == 0:
                        continue
                    else:
                        self.pairs.append(row)
        else:
            with open(csv_file, 'r') as f:
                for line in f.readlines():
                    self.pairs.append(line.strip().split(' '))

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):

        img_path1, img_path2, match = self.pairs[idx]
        img1 = Image.open(os.path.join(self.data_dir, img_path1))
        img2 = Image.open(os.path.join(self.data_dir, img_path2))
        return self.transform(img1), self.transform(img2), int(match)

In [ ]:
class TestImagePairDataset(torch.utils.data.Dataset):

    def __init__(self, data_dir, csv_file, transform):
        self.data_dir = data_dir
        self.transform = transform
        self.pairs = []
        if csv_file.endswith('.csv'):
            with open(csv_file, 'r') as f:
                reader = csv.reader(f)
                for i, row in enumerate(reader):
                    if i == 0:
                        continue
                    else:
                        self.pairs.append(row)
        else:
            with open(csv_file, 'r') as f:
                for line in f.readlines():
                    self.pairs.append(line.strip().split(' '))

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):

        img_path1, img_path2 = self.pairs[idx]
        img1 = Image.open(os.path.join(self.data_dir, img_path1))
        img2 = Image.open(os.path.join(self.data_dir, img_path2))
        return self.transform(img1), self.transform(img2)

## Create Dataloaders for Image Recognition

### For Dataloader for image Recognition I modified training datasets  where I included data augmentation, Resizing of image, color Jitter, and kept normalizing as it was given. This transformation helped to enhance  the diversity of the training data and helping the model in learning.

### Kept the  Validation transform and train and validation loader as they were given.

In [ ]:
data_dir = config['data_dir']
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'dev')
os.makedirs(config['checkpoint_dir'], exist_ok=True)

# Train transforms
train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize(112),                     # Image Resizing 
    torchvision.transforms.RandomPerspective(distortion_scale=0.3, p=0.5), 
    torchvision.transforms.ColorJitter(brightness=0.2), 
    torchvision.transforms.RandomRotation(degrees=25),   
    torchvision.transforms.RandomHorizontalFlip(p=0.25),     # Image Fliping
    torchvision.transforms.RandAugment(num_ops=4),          # Random Augmentation
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                     std=[0.5, 0.5, 0.5])  # Normalize the image
])

# Validation transforms 
val_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize(112),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                     std=[0.5, 0.5, 0.5])
])

# Get Datasets
train_dataset = torchvision.datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset = torchvision.datasets.ImageFolder(val_dir, transform=val_transforms)

# Data loaders
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=config["batch_size"],
                                           shuffle=True,
                                           pin_memory=True,
                                           num_workers=8)
val_loader = torch.utils.data.DataLoader(val_dataset,
                                         batch_size=config["batch_size"],
                                         shuffle=False,
                                         num_workers=4)


In [ ]:
data_dir = config['data_ver_dir']

pairs_dir = config['pair_dir']
# get datasets
val_dataset = os.path.join(config['data_ver_dir'], 'val_data')

# TODO: Add your validation pair txt file
val_pairs = os.path.join(pairs_dir, 'val_pairs.txt') 
pair_dataset = ImagePairDataset(data_dir, csv_file=val_pairs, transform=val_transforms)
pair_dataloader = torch.utils.data.DataLoader(pair_dataset,
                                              batch_size=config["batch_size"],
                                              shuffle=False,
                                              pin_memory=True,
                                              num_workers=4)
test_data = os.path.join(config['data_ver_dir'], 'test_data')
# TODO: Add your validation pair txt file
test_pairs = os.path.join(pairs_dir, 'test_pairs.txt')
test_pair_dataset = TestImagePairDataset(data_dir, csv_file= test_pairs, transform=val_transforms)
test_pair_dataloader = torch.utils.data.DataLoader(test_pair_dataset,
                                              batch_size=config["batch_size"],
                                              shuffle=False,
                                              pin_memory=True,
                                              num_workers=4)

# EDA and Viz

In [ ]:
# Double-check your dataset/dataloaders work as expected

print("Number of classes    : ", len(train_dataset.classes))
print("No. of train images  : ", train_dataset.__len__())
print("Shape of image       : ", train_dataset[0][0].shape)
print("Batch size           : ", config['batch_size'])
print("Train batches        : ", train_loader.__len__())
print("Val batches          : ", val_loader.__len__())

# Feel free to print more things if needed

In [ ]:
# Visualize a few images in the dataset

"""
You can write your own code, and you don't need to understand the code
It is highly recommended that you visualize your data augmentation as sanity check
"""

r, c    = [5, 5]
fig, ax = plt.subplots(r, c, figsize= (15, 15))

k       = 0
dtl     = torch.utils.data.DataLoader(
    dataset     = torchvision.datasets.ImageFolder(train_dir, transform= train_transforms), # dont wanna see the images with transforms
    batch_size  = config['batch_size'],
    shuffle     = True)

for data in dtl:
    x, y = data

    for i in range(r):
        for j in range(c):
            img = x[k].numpy().transpose(1, 2, 0)
            ax[i, j].imshow(img)
            ax[i, j].axis('off')
            k+=1
    break

del dtl

# Model Architecture

FAQ:

**What's a very low early deadline architecture (mandatory early submission)**?

- The very low early deadline architecture is a 5-layer CNN. Keep in mind the parameter limit is 18M.
- The first convolutional layer has 64 channels, kernel size 7, and stride 4. The next three have 128, 256, 512 and 1024 channels. Each have kernel size 3 and stride 2. Documentation to make convolutional layers: https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
- Think about strided convolutions from the lecture, as convolutions with stride = 1 and downsampling. For strided convolution, what padding do you need for preserving the spatial resolution? (Hint => padding = kernel_size // 2) - Think why?
- Each convolutional layer is accompanied by a Batchnorm and ReLU layer.
- Finally, you want to average pool over the spatial dimensions to reduce them to 1 x 1. Use AdaptiveAvgPool2d. Documentation for AdaptiveAvgPool2d: https://pytorch.org/docs/stable/generated/torch.nn.AdaptiveAvgPool2d.html
- Then, remove (Flatten?) these trivial 1x1 dimensions away.
Look through https://pytorch.org/docs/stable/nn.html


**Why does a very simple network have 4 convolutions**?

Input images are 112x112. Note that each of these convolutions downsample. Downsampling 2x effectively doubles the receptive field, increasing the spatial region each pixel extracts features from. Downsampling 32x is standard for most image models.

**Why does a very simple network have high channel sizes**?

Every time you downsample 2x, you do 4x less computation (at same channel size). To maintain the same level of computation, you 2x increase # of channels, which increases computation by 4x. So, balances out to same computation. Another intuition is - as you downsample, you lose spatial information. We want to preserve some of it in the channel dimension.

**What is return_feats?**

It essentially returns the second-to-last-layer features of a given image. It's a "feature encoding" of the input image, and you can use it for the verification task. You would use the outputs of the final classification layer for the classification task. You might also find that the classification outputs are sometimes better for verification too - try both.

In [ ]:
# # TODO: Fill out the model definition below

# class Network(torch.nn.Module):

#     def __init__(self, num_classes=8631):
#         super().__init__()

#         self.backbone = torch.nn.Sequential(
#             # TODO
#             )

#         self.cls_layer = #TODO

#     def forward(self, x):
#             # TODO:

#         return {"feats": feats, "out": out}

# # Initialize your model
# model = Network().to(DEVICE)
# summary(model, (3, 112, 112))


In [ ]:
pip install torchsummary

In [ ]:
from torchsummary import summary

### I expermented with different network and Residual Network Squeeze and Excite is the one yield the high cutoff of the homework. where others were performing very poorly in image recognitionn not even giving the low cutoff.
### - For the class of Squeeze and Excite I utilized global average pooling and and two fuly connected layer
### - for Residual Squeeze and Excite I employed two convolution and applyied batch normalization to prevent overfitting and improve generalization process, and utilized the ReLU as activation function.
### - Network class with convolution layer, batch Normalization,and ReLU as activation function, For effective feature extraction and stable training

In [ ]:
class SExcite(torch.nn.Module):
    def __init__(self, channels, reduction=16):
        super(SExcite, self).__init__()
        self.fc1 = torch.nn.Linear(channels, channels // reduction, bias=False)
        self.fc2 = torch.nn.Linear(channels // reduction, channels, bias=False)
    
    def forward(self, x):
        batch, channels, _, _ = x.size()
        y = x.view(batch, channels, -1).mean(dim=2)  # Global average pooling
        y = F.relu(self.fc1(y))
        y = torch.sigmoid(self.fc2(y))
        y = y.view(batch, channels, 1, 1)
        return x * y

class ReSExcite(torch.nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None, reduction=16):
        super(ReSExcite, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = torch.nn.BatchNorm2d(out_channels)
        self.conv2 = torch.nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = torch.nn.BatchNorm2d(out_channels)
        self.se = SExcite(out_channels, reduction)
        self.downsample = downsample
        self.relu = torch.nn.ReLU(inplace=True)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.se(out)

        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        return self.relu(out)

class Network(torch.nn.Module):
    def __init__(self, num_classes=8631):
        super(Network, self).__init__()
        self.backbone = torch.nn.Sequential(
            torch.nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            self._make_layer(64, 64, 3),
            self._make_layer(64, 128, 4, stride=2),
            self._make_layer(128, 256, 6, stride=2),
            self._make_layer(256, 512, 3, stride=2),
            torch.nn.AdaptiveAvgPool2d((1, 1))
        )
        self.cls_layer = torch.nn.Linear(512, num_classes)

    def _make_layer(self, in_channels, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or in_channels != out_channels:
            downsample = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                torch.nn.BatchNorm2d(out_channels),
            )

        layers = [ReSExcite(in_channels, out_channels, stride, downsample)]
        for _ in range(1, blocks):
            layers.append(ReSExcite(out_channels, out_channels))
        return torch.nn.Sequential(*layers)

    def forward(self, x):
        feats = self.backbone(x)
        feats = feats.view(feats.size(0), -1)
        out = self.cls_layer(feats)
        return {"feats": feats, "out": out}

# Initialize the model
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = Network().to(DEVICE)

# Summary of the model
summary(model, (3, 112, 112))


### For Criterion I used CrossEntropyLoss, and about optimizer I experiment on Adam, AdamW, and SGD is the one that gave the good EER compare to other optimizer with weight decay for regulerizing of 0.001. and for scheduler I used CossineAnnealingLR.

In [ ]:
# --------------------------------------------------- #

# Defining Loss function
#criterion = # TODO: What loss do you need for a multi class classification problem and would label smoothing be beneficial here?
criterion = torch.nn.CrossEntropyLoss()

# --------------------------------------------------- #

# Defining Optimizer
#optimizer =  # TODO: Feel free to pick a optimizer

# --------------------------------------------------- #
import torch.optim as optim

# Define the SGD optimizer
optimizer = optim.SGD(model.parameters(), lr=config['lr'], momentum=0.9, weight_decay=1e-3       
)

# Optionally, define a learning rate scheduler

# Defining Scheduler
#scheduler = None# TODO: Use a good scheduler such as ReduceLRonPlateau, StepLR, MultistepLR, CosineAnnealing, etc.
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=60, eta_min=1e-6)

# --------------------------------------------------- #

# Initialising mixed-precision training. # Good news. We've already implemented FP16 (Mixed precision training) for you
# It is useful only in the case of compatible GPUs such as T4/V100
scaler = torch.cuda.amp.GradScaler()

# Metrics

In [ ]:
class AverageMeter:
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    maxk = min(max(topk), output.size()[1])
    batch_size = target.size(0)
    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.reshape(1, -1).expand_as(pred))
    return [correct[:min(k, maxk)].reshape(-1).float().sum(0) * 100. / batch_size for k in topk]

In [ ]:
def get_ver_metrics(labels, scores, FPRs):
    # eer and auc
    fpr, tpr, _ = mt.roc_curve(labels, scores, pos_label=1)
    roc_curve = interp1d(fpr, tpr)
    EER = 100. * brentq(lambda x : 1. - x - roc_curve(x), 0., 1.)
    AUC = 100. * mt.auc(fpr, tpr)

    # get acc
    tnr = 1. - fpr
    pos_num = labels.count(1)
    neg_num = labels.count(0)
    ACC = 100. * max(tpr * pos_num + tnr * neg_num) / len(labels)

    # TPR @ FPR
    if isinstance(FPRs, list):
        TPRs = [
            ('TPR@FPR={}'.format(FPR), 100. * roc_curve(float(FPR)))
            for FPR in FPRs
        ]
    else:
        TPRs = []

    return {
        'ACC': ACC,
        'EER': EER,
        'AUC': AUC,
        'TPRs': TPRs,
    }

# Train and Validation Function

In [ ]:
def train_epoch(model, dataloader, optimizer, lr_scheduler, scaler, device, config):

    model.train()

    # metric meters
    loss_m = AverageMeter()
    acc_m = AverageMeter()

    # Progress Bar
    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train', ncols=5)

    for i, (images, labels) in enumerate(dataloader):

        optimizer.zero_grad() # Zero gradients

        # send to cuda
        images = images.to(device, non_blocking=True)
        if isinstance(labels, (tuple, list)):
            targets1, targets2, lam = labels
            labels = (targets1.to(device), targets2.to(device), lam)
        else:
            labels = labels.to(device, non_blocking=True)

        # forward
        with torch.cuda.amp.autocast():  # This implements mixed precision. Thats it!
            outputs = model(images)

            # Use the type of output depending on the loss function you want to use
            loss = criterion(outputs['out'], labels)

        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update()
        # metrics
        loss_m.update(loss.item())
        if 'feats' in outputs:
            acc = accuracy(outputs['out'], labels)[0].item()
        else:
            acc = 0.0
        acc_m.update(acc)

        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            # acc         = "{:.04f}%".format(100*accuracy),
            acc="{:.04f}% ({:.04f})".format(acc, acc_m.avg),
            loss        = "{:.04f} ({:.04f})".format(loss.item(), loss_m.avg),
            lr          = "{:.04f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

    # You may want to call some schedulers inside the train function. What are these?
    #if lr_scheduler is not None:
     #   lr_scheduler.step()

    batch_bar.close()

    return acc_m.avg, loss_m.avg

In [ ]:
@torch.no_grad()
def valid_epoch_cls(model, dataloader, device, config):

    model.eval()
    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=False, desc='Val Cls.', ncols=5)

    # metric meters
    loss_m = AverageMeter()
    acc_m = AverageMeter()

    for i, (images, labels) in enumerate(dataloader):

        # Move images to device
        images, labels = images.to(device), labels.to(device)

        # Get model outputs
        with torch.inference_mode():
            outputs = model(images)
            loss = criterion(outputs['out'], labels)

        # metrics
        acc = accuracy(outputs['out'], labels)[0].item()
        loss_m.update(loss.item())
        acc_m.update(acc)

        batch_bar.set_postfix(
            acc         = "{:.04f}% ({:.04f})".format(acc, acc_m.avg),
            loss        = "{:.04f} ({:.04f})".format(loss.item(), loss_m.avg))

        batch_bar.update()

    batch_bar.close()
    return acc_m.avg, loss_m.avg

In [ ]:
gc.collect() # These commands help you when you face CUDA OOM error
torch.cuda.empty_cache()

# Verification Task

In [ ]:
def valid_epoch_ver(model, pair_data_loader, device, config):

    model.eval()
    scores = []
    match_labels = []
    batch_bar = tqdm(total=len(pair_data_loader), dynamic_ncols=True, position=0, leave=False, desc='Val Veri.')
    for i, (images1, images2, labels) in enumerate(pair_data_loader):

        # match_labels = match_labels.to(device)
        images = torch.cat([images1, images2], dim=0).to(device)
        # Get model outputs
        with torch.inference_mode():
            outputs = model(images)

        feats = F.normalize(outputs['feats'], dim=1)
        feats1, feats2 = feats.chunk(2)
        similarity = F.cosine_similarity(feats1, feats2)
        scores.append(similarity.cpu().numpy())
        match_labels.append(labels.cpu().numpy())
        batch_bar.update()

    scores = np.concatenate(scores)
    match_labels = np.concatenate(match_labels)

    FPRs=['1e-4', '5e-4', '1e-3', '5e-3', '5e-2']
    metric_dict = get_ver_metrics(match_labels.tolist(), scores.tolist(), FPRs)
    print(metric_dict)

    return metric_dict['ACC'], metric_dict['EER']




# WandB

In [ ]:
wandb.login(key="65ab01e861811be49c0d4f76312a0729eb26cdfc") # API Key is in your wandb account, under settings (wandb.ai/settings)

### For the other Runs I resumed the model with the best verifacation since it was corresponding to the lowest EER. The name  of my wandb runs include High-Resumerunning2:11-10-2024-SERSNET-Run3SGDFinal, High Cutoff Submission, Trial Run with dates and others.  For Wandb we used one account as team where we monitered the run and help each other. all my runs does not have my name on them compare to others. I resumed on notebook 2 times but its not what i selected which is why code for resuming is comment

### On epoch 50 I get the best EER and validation retrival
### Epoch 50/70:
## 499 Train Cls. Acc 78.3812%  Train Cls. Loss 1.2145  Learning Rate 0.0001
### 500 Val Cls. Acc 80.0565%  Val Cls. Loss 1.0916
### 501 {'ACC': 90.4, 'EER': 9.775967413441961, 'AUC': 96.0667256191006, 'TPRs': [('TPR@FPR=1e-4', 44.806517311608964), ('TPR@FPR=5e-4', 44.806517311608964), ('TPR@FPR=1e-3', 44.806517311608964), ####('TPR@FPR=5e-3', 47.45417515274949), ('TPR@FPR=5e-2', 82.68839103869654)]}
### 502 Val Ret. Acc 90.4000%
### c503 Saved best classification model with Val Cls. Acc: 80.0565%

In [ ]:
# Create your wandb run
run = wandb.init(
    name = "12-10-2024 Trial run60 Epoch", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    #id = 'g4v1lxiq',### Insert specific run id here if you want to resume a previous run
    #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw2p2-ablations", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)
# Restore the best retrieval model checkpoint
#Res_model = wandb.restore('checkpoint/best_ret.pth')
#checkpoint = torch.load(Res_model.name)

# Load only the model state dict from the checkpoint
#model.load_state_dict(checkpoint["model_state_dict"])

# Checkpointing and Loading Model

In [ ]:
# Uncomment the line for saving the scheduler save dict if you are using a scheduler
def save_model(model, optimizer, scheduler, metrics, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         'metric'                   : metrics,
         'epoch'                    : epoch},
         path)


def load_model(model, optimizer=None, scheduler=None, path='./checkpoint.pth'):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    else:
        optimizer = None
    if scheduler is not None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    else:
        scheduler = None
    epoch = checkpoint['epoch']
    metrics = checkpoint['metric']
    return model, optimizer, scheduler, epoch, metrics

# Experiments

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def test_epoch_ver(model, pair_data_loader, config):

    model.eval()
    scores = []
    batch_bar = tqdm(total=len(pair_data_loader), dynamic_ncols=True, position=0, leave=False, desc='Val Veri.')
    for i, (images1, images2) in enumerate(pair_data_loader):

        images = torch.cat([images1, images2], dim=0).to(DEVICE)
        # Get model outputs
        with torch.inference_mode():
            outputs = model(images)

        feats = F.normalize(outputs['feats'], dim=1)
        feats1, feats2 = feats.chunk(2)
        similarity = F.cosine_similarity(feats1, feats2)
        scores.extend(similarity.cpu().numpy().tolist())
        batch_bar.update()

    return scores

### For the training loop, I included the code to moniter the EER on wandb chart and also be able to save the best verification model and its corresponding CSV file on wandb. 
### For this run I initiated 70 epoch but my kaggle crashed when it was running the 63th epoch and since I added  the code to retrieve the csv of best model,
### I stopped there where I used RESSE Network, SGD as optimizer and CosineAnealing as scheduler

In [ ]:
e = 0
best_valid_cls_acc = 0.0
eval_cls = True
best_valid_ret_acc = 0.0
for epoch in range(e, config['epochs']):
    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    # Train
    train_cls_acc, train_loss = train_epoch(model, train_loader, optimizer, scheduler, scaler, DEVICE, config)
    curr_lr = float(optimizer.param_groups[0]['lr'])
    print("\nEpoch {}/{}: \nTrain Cls. Acc {:.04f}%\t Train Cls. Loss {:.04f}\t Learning Rate {:.04f}".format(
        epoch + 1, config['epochs'], train_cls_acc, train_loss, curr_lr))

    # Metrics for tracking
    metrics = {
        'train_cls_acc': train_cls_acc,
        'train_loss': train_loss,
    }

    # Validation (classification)
    if eval_cls:
        valid_cls_acc, valid_loss = valid_epoch_cls(model, val_loader, DEVICE, config)
        print("Val Cls. Acc {:.04f}%\t Val Cls. Loss {:.04f}".format(valid_cls_acc, valid_loss))
        metrics.update({
            'valid_cls_acc': valid_cls_acc,
            'valid_loss': valid_loss,
        })

    # Validation (retrieval)
    valid_ret_acc, valid_eer = valid_epoch_ver(model, pair_dataloader, DEVICE, config)
    print("Val Ret. Acc {:.04f}%".format(valid_ret_acc))
    metrics.update({
        'valid_ret_acc': valid_ret_acc,
        'valid_eer': valid_eer
    })
    

    # Save the best classification model only
    if eval_cls and valid_cls_acc >= best_valid_cls_acc:
        best_valid_cls_acc = valid_cls_acc
        best_model_path = os.path.join(config['checkpoint_dir'], 'best_model.pth')
        torch.save(model.state_dict(), best_model_path)
        wandb.save(best_model_path, base_path=config['checkpoint_dir'])  # Ensure the correct base path is used
        
        print(f"Saved best classification model with Val Cls. Acc: {valid_cls_acc:.04f}%")
    if valid_ret_acc >= best_valid_ret_acc:
        best_valid_ret_acc = valid_ret_acc
        save_model(model, optimizer, scheduler, metrics, epoch, os.path.join(config['checkpoint_dir'], 'best_ret_final.pth'))
        wandb.save(os.path.join(config['checkpoint_dir'], 'best_ret_final.pth'))
        print("Saved best retrieval model")        
        scores = test_epoch_ver(model, test_pair_dataloader, config)
        csv_path = "Final_submission_best_ret_epoch_.csv"
        with open(csv_path, "w+") as f:
            f.write("ID,Label\n")
            for i in range(len(scores)):
                f.write("{},{}\n".format(i, scores[i]))
        wandb.save(csv_path)
        print("Saved CSV for best retrieval model at epoch ")
        
    

    # Log metrics to WandB
    if run is not None:
        run.log(metrics)

    # Learning rate scheduler step
    if scheduler is not None:
        if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
            scheduler.step(valid_loss)
        else:
            scheduler.step()

print('Completed!')


# Testing and Kaggle Submission (Verification)

In [ ]:
scores = test_epoch_ver(model, test_pair_dataloader, config)

In [ ]:
with open("Final_submission_1.csv", "w+") as f:
    f.write("ID,Label\n")
    for i in range(len(scores)):
        f.write("{},{}\n".format(i, scores[i]))

Saving the CSV or prediction in the output of my kaggle for later use.

## Here is the Link to Wandb study team :
# https://api.wandb.ai/links/angeizabayo05-carnegie-mellon-university/wbhievnv
### For this homework we tested and experimented different technique in order to achieve the required output and as shown in the link we did many runs and the one yield or returned the high cutoff score is "12-10-2024 Trial run60 Epoch"